# Sample Queries on the 1000 Genomes, gnomAD and ClinVar data lake

In this notebook, we will demonstrate some sample genomics queries that are typically made by clinical geneticists and researchers on genomics variant data. We will use the parquet/ORC transformed variant data from the 3502 DRAGEN-reanalyzed 1000 Genomes dataset available in 3 different schemas at s3://aws-roda-hcls-datalake/thousandgenomes_dragen/:

1. var_partby_samples - Dataset partitioned by sample ID
2. var_partby_chrom - Dataset partitioned by chromosome and bucketed by samples
3. var_nested - Nested schema consisting of variant sites with sample IDs and genotypes that contain the variant


We use the transformed annotations from ClinVar that are available at https://registry.opendata.aws/clinvar/ and the transformed population allele frequency data from gnomAD at to demonstrate how to make queries that use the raw variant data with annotations. 

Please use the CloudFormation templates for 1000 Genomes DRAGEN, ClinVar and gnomAD available at https://github.com/aws-samples/data-lake-as-code/tree/roda to add these tables to your Glue Data Catalog.

We use the annotations from ClinVar that are available at https://registry.opendata.aws/clinvar/ and the population allele frequency data from gnomAD to demonstrate how to make queries that use the raw variant data with annotations. 

## Import Dependencies

In [181]:
import boto3, os
import pandas as pd

s3 = boto3.resource('s3')
glue = boto3.client('glue')
cfn = boto3.client('cloudformation')
import sys
!{sys.executable} -m pip install PyAthena


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [182]:
session = boto3.session.Session()
region = session.region_name
print(region)

us-east-1


In [183]:
databasename="\"thousandgenomes_dragen_dl_awsroda\""
partitioned_chr = "\"var_partby_chrom\""
partitioned_samples = "\"var_partby_samples\""
nested = "\"var_nested\""
clinvarname="\"clinvar_summary_variants_dl-awsroda\""
clinvar_summary="\"variant_summary\""

### Connect to Athena

In [196]:
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.async_cursor import AsyncCursor, AsyncDictCursor
from pyathena.error import NotSupportedError, ProgrammingError
from pyathena.model import AthenaQueryExecution
from pyathena.result_set import AthenaResultSet

conn = connect(s3_staging_dir='s3://athena-query-results-ss',region_name=region) #replace with your own bucket name
cursor = conn.cursor()

def execute_query_async(query):
    query_summary = '''Query execution summary:
        DataScanned(MB): {}
        ExecutionTime(s): {}
        QueuingTime(s): {}'''

    df = None
    acursor = conn.cursor(AsyncCursor)
    query_id, future = acursor.execute(query)
    result_set = future.result()
    if result_set.state == AthenaQueryExecution.STATE_SUCCEEDED:
        print(query_summary.format(result_set.data_scanned_in_bytes/100000, 
                                   result_set.engine_execution_time_in_millis/1000, 
                                   result_set.query_queue_time_in_millis/1000))    
        rows = result_set.fetchall()
        cols = [x[0] for x in result_set.description]
        df = pd.DataFrame(rows, columns=cols)
        
    acursor.close()
    return df

#### Let us explore the schema of the tables under the 1000 genomes transformed dataset

Here is the flat table schema for the data sorted by sample ID

In [185]:
cursor.execute("SELECT * from " + databasename + "." + partitioned_samples + "limit 10")
df = as_pandas(cursor)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   variant_id                     10 non-null     object 
 1   chrom                          10 non-null     object 
 2   pos                            10 non-null     int64  
 3   alleles                        10 non-null     object 
 4   rsid                           0 non-null      object 
 5   qual                           10 non-null     float64
 6   filters                        10 non-null     object 
 7   info.ac                        10 non-null     object 
 8   info.af                        10 non-null     object 
 9   info.an                        10 non-null     int64  
 10  info.db                        10 non-null     bool   
 11  info.dp                        10 non-null     int64  
 12  info.end                       0 non-null      object

Here is the schema for the flat table partitioned by chromosome and bucketed by samples. As you can see, the schema is very similar to the one partitioned by samples, except that "chrom" is the field that the data is partitioned by

In [186]:
cursor.execute("SELECT * from " + databasename + "." + partitioned_chr + "limit 10")
df = as_pandas(cursor)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   variant_id                     10 non-null     object 
 1   pos                            10 non-null     int64  
 2   ref                            10 non-null     object 
 3   alt                            10 non-null     object 
 4   sample_id                      10 non-null     object 
 5   alleles                        10 non-null     object 
 6   rsid                           0 non-null      object 
 7   qual                           10 non-null     float64
 8   filters                        10 non-null     object 
 9   info.ac                        0 non-null      object 
 10  info.af                        0 non-null      object 
 11  info.an                        0 non-null      object 
 12  info.db                        10 non-null     bool  

Here is the nested schema. In this schema, most of the FORMAT and INFO fields are not retained. Each row is a variant site with an array consisting of sample IDs and genotypes

In [187]:
cursor.execute("SELECT * from " + databasename +"." + nested + " limit 10")
df = as_pandas(cursor)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   variant_id  10 non-null     object
 1   pos         10 non-null     int64 
 2   ref         10 non-null     object
 3   alt         10 non-null     object
 4   samples     10 non-null     object
 5   chrom       10 non-null     object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


,variant_id,pos,ref,alt,samples,chrom
0,chrY:CT:C:3526296,3526296,CT,C,"[{id=NA18504, gts=[1]}, {id=HG03812, gts=[1]},...",chrY
1,chrY:T:C:3749071,3749071,T,C,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
2,chrY:C:G:3992262,3992262,C,G,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
3,chrM:T:C:195,195,T,C,"[{id=HG01573, gts=[0, 1]}, {id=HG03850, gts=[0...",chrM
4,chrM:C:A:16159,16159,C,A,"[{id=HG01573, gts=[0, 1]}, {id=HG01928, gts=[0...",chrM
5,chrM:C:CA:16257,16257,C,CA,"[{id=HG01573, gts=[1, 0]}, {id=HG00844, gts=[0...",chrM
6,chrM:G:A:8363,8363,G,A,"[{id=HG00149, gts=[0, 1]}, {id=NA18987, gts=[0...",chrM
7,chrM:A:G:11818,11818,A,G,"[{id=HG03650, gts=[0, 1]}, {id=NA19771, gts=[0...",chrM
8,chrM:C:T:16222,16222,C,T,"[{id=HG03650, gts=[0, 1]}, {id=HG04094, gts=[0...",chrM
9,chrM:A:G:5951,5951,A,G,"[{id=HG03850, gts=[0, 1]}, {id=HG01986, gts=[0...",chrM


### Queries on variant data alone
Here are some examples of queries on just the variant data with no annotations

**Example 1:** Query for variants that are on a specific gene BRCA1 (chr17:43044295-43125364) in a specific sample HG02625.
This query will run faster and will need to scan only data within the specific sample partition, so we will use the dataset that is partitioned by sample

In [188]:
query = """SELECT * from {}.{}
  WHERE chrom='chr17'
  AND pos BETWEEN 43044295 AND 43125364 
  AND sample_id = 'HG02625' """.format(databasename,partitioned_samples)

df = execute_query_async(query)
df

Query execution summary:
        DataScanned: 47353926
        ExecutionTime(s): 8.382
        QueuingTime(s): 0.156


,variant_id,chrom,pos,alleles,rsid,qual,filters,info.ac,info.af,info.an,...,mb,pl,pri,ps,sb,sq,sample_id,ref,alt,partition_0
0,chr17:G:A:43044391,chr17,43044391,"[G, A]",None,45.73,[],[1],[0.5],2,...,"[12, 17, 9, 10]","[81, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[13, 16, 13, 6]",None,HG02625,G,A,HG02625
1,chr17:CTT:C:43044804,chr17,43044804,"[CTT, C, CTTTT]",None,308.73,[],"[1, 1]","[0.5, 0.5]",2,...,"[0, 0, 17, 10]","[317, 263, 52, 391, 0, 54]","[0.0, 4.0, 7.0, 4.0, 8.0, 7.0]",NaN,"[0, 0, 17, 10]",None,HG02625,CTT,C,HG02625
2,chr17:C:A:43045257,chr17,43045257,"[C, A]",None,49.94,[],[1],[0.5],2,...,"[9, 10, 11, 7]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[12, 7, 8, 10]",None,HG02625,C,A,HG02625
3,chr17:A:G:43046604,chr17,43046604,"[A, G]",None,48.94,[],[1],[0.5],2,...,"[9, 8, 5, 8]","[84, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[11, 6, 9, 4]",None,HG02625,A,G,HG02625
4,chr17:C:CA:43046757,chr17,43046757,"[C, CA]",None,50.00,[],[1],[0.5],2,...,"[8, 5, 9, 6]","[56, 0, 50]","[0.0, 6.0, 9.0]",NaN,"[7, 6, 10, 5]",None,HG02625,C,CA,HG02625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,chr17:GTT:G:43123349,chr17,43123349,"[GTT, G]",None,50.00,[],[1],[0.5],2,...,"[3, 5, 7, 3]","[52, 0, 50]","[0.0, 2.0, 5.0]",NaN,"[5, 3, 4, 6]",None,HG02625,GTT,G,HG02625
207,chr17:A:G:43123628,chr17,43123628,"[A, G]",None,47.93,[],[1],[0.5],2,...,"[10, 12, 6, 10]","[83, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[12, 10, 8, 8]",None,HG02625,A,G,HG02625
208,chr17:A:G:43124230,chr17,43124230,"[A, G]",None,50.00,[],[1],[0.5],2,...,"[9, 7, 6, 12]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[8, 8, 13, 5]",None,HG02625,A,G,HG02625
209,chr17:T:C:43124331,chr17,43124331,"[T, C]",None,50.00,[],[1],[0.5],2,...,"[6, 5, 6, 6]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[10, 1, 8, 4]",None,HG02625,T,C,HG02625


**Example 2:** Query to find samples that have variants in a specific region, in this case, the BRCA1 gene (chr17:43044295-43125364). 
This query can be run on the data that is partitioned by chromosome 

In [189]:
query = """SELECT DISTINCT(sample_id) from {}.{}
  where chrom='chr17' 
  and pos between 43044295 and 43125364 """.format(databasename,partitioned_chr)

df=execute_query_async(query)
df

Query execution summary:
        DataScanned: 915862188
        ExecutionTime(s): 2.376
        QueuingTime(s): 0.158


,sample_id
0,NA19983
1,NA18968
2,HG03401
3,HG03190
4,HG02075
...,...
3197,HG01798
3198,HG00663
3199,HG03897
3200,NA20870


**Example 2a:** Query 2 on the nested data

In [197]:
query = """SELECT DISTINCT(sample.id) from (
  SELECT samples from {}.{}
  where chrom='chr1'
  and pos between 9033567 and 9142000
  ) as f,
  unnest(f.samples) as s(sample)
  order by 1""".format(databasename,nested)
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 14821.13335
        ExecutionTime(s): 7.328
        QueuingTime(s): 0.164


,id
0,HG00096
1,HG00097
2,HG00099
3,HG00100
4,HG00101
...,...
3197,NA21137
3198,NA21141
3199,NA21142
3200,NA21143


**Example 3:** Query to find only samples that have homozygous variants in a specific region, in this case, the BRCA1 gene (chr17:43044295-43125364). 

In [198]:
query = """SELECT DISTINCT(sample_id) from {}.{}
  where chrom='chr17'
  and pos between 43044295 and 43125364
  and array_join("gt.alleles", '|') in ('0|1')""".format(databasename,partitioned_chr)

df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 10959.8672
        ExecutionTime(s): 2.643
        QueuingTime(s): 0.305


,sample_id
0,HG03633
1,NA07034
2,HG00280
3,NA18566
4,HG03846
...,...
3179,HG04204
3180,NA20756
3181,HG00561
3182,HG01668


### Queries joining variant data with ClinVar annotation data

**ClinVar has a number of tables, but the one that we will use here is the summary_variants table. This table has many fields, but we will only be using a subset of them. We also need to create a Variant ID that can be used to join with the raw variant tables, so to make this easier, we will create a "view" of the summary_variants table that we can use with subsequent queries**

In [199]:
cursor.execute("create or replace view clinvar as "
       "select concat('chr', chromosome, ':', referenceallelevcf, ':', alternateallelevcf, ':', cast(positionvcf as varchar)) as variant_id, "
       "chromosome as chrom, "
       "positionvcf as pos, "
       "referenceallelevcf as ref, "
       "alternateallelevcf as alt, "
       "genesymbol as genename, " 
       "clinicalsignificance as clinicalsignificance, "
       "numbersubmitters as num_submitters, "
       "reviewstatus as reviewstatus, "
       "split(phenotypeids, ',') as phenotypeids, "
       "split(phenotypelist, ',') as phenotypelist "
       "from \"clinvar_summary_variants_dl-awsroda\".\"variant_summary\" where assembly = 'GRCh38'"
        "and referenceallelevcf <> 'na' and alternateallelevcf <> 'na'")


**Example 4:** Modify query 3 to find the number of samples that have "variants of Unknown Significance" in the BRCA1 gene

In [200]:
query = """SELECT count(DISTINCT(sample.id)) FROM (
       SELECT samples FROM {}.{} AS v 
       JOIN clinvar a 
       ON v.variant_id = a.variant_id 
       WHERE a.clinicalsignificance = 'Uncertain significance' 
       AND v.chrom='chr17' 
       AND v.pos BETWEEN 43044295 AND 43125364 
       ) AS f, 
       UNNEST(f.samples) AS s(sample) """.format(databasename, nested)
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 6687.40946
        ExecutionTime(s): 5.725
        QueuingTime(s): 0.203


,_col0
0,1550


**Example 4a:** To have a greater degree of confidence in the results, we want to only consider those entries in ClinVar that have more than 1 submission. This query will filter the results from Example 4 to only consider those that have > 1 submitter.

In [201]:
query = """SELECT count(DISTINCT(sample.id)) from (
       SELECT samples FROM {}.{} as v 
       JOIN clinvar a
       ON v.variant_id = a.variant_id
       WHERE a.clinicalsignificance = 'Uncertain significance' 
       AND v.chrom='chr17' 
       AND v.pos BETWEEN 43044295 AND 43125364 
       AND a.num_submitters > 1 
       ) AS f, 
       UNNEST(f.samples) AS s(sample)  """.format(databasename, nested)
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 6694.43858
        ExecutionTime(s): 5.143
        QueuingTime(s): 0.151


,_col0
0,41


**Example 5a:**  Find all variants in this dataset that have a pathogenic variant in the BRCA1 gene (using the gene annotation from ClinVar). Running the same query using the chromosome and position range for the BRCA1 gene is much faster due to the partitioning of the data.

In [208]:
query = """SELECT COUNT(DISTINCT sample_id) FROM {}.{} as v 
           JOIN clinvar AS a  
           ON v.variant_id = a.variant_id 
           WHERE a.genename='BRCA1' AND clinicalsignificance='Pathogenic' """.format(databasename,partitioned_chr)

df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 924640.11515
        ExecutionTime(s): 79.597
        QueuingTime(s): 0.154


,_col0
0,4


**Example 5b:** Run Query 5a using the nested dataset

In [209]:
query = """SELECT COUNT(DISTINCT sample.id) FROM 
           (SELECT samples from {}.{} AS v  
           JOIN clinvar AS a 
           ON v.variant_id = a.variant_id 
           WHERE a.genename='BRCA1' AND clinicalsignificance='Pathogenic') AS f, 
           UNNEST(f.samples) AS s(sample)""".format(databasename, nested)

df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 230663.85143
        ExecutionTime(s): 35.196
        QueuingTime(s): 0.227


,_col0
0,4


### Queries using 1000 genomes with gnomAD and ClinVar

The gnomAD sites data is available at s3://juayu-sampledata/geno_dataset/gnomad/sites/. We will first create a View of the gnomAD table with the fields we are going to be using in this set of queries to simplify things. 

In [204]:
cursor.execute("create or replace view gnomad as "
       " select concat(\"locus.contig\", ':', alleles[1], ':', alleles[2], ':', cast(\"locus.position\" as varchar)) as variant_id "
       ", \"locus.contig\" as chrom"
       ", \"locus.position\" as pos"
       ", alleles[1] as ref"
       ", alleles[2] as alt"
       ", rsid" 
       ", filters"
       ", \"info.ac\" as info_ac"
       ", \"info.an\" as info_an"
       ", \"info.af\" as info_af"
       ", \"info.popmax\" as info_popmax"
       ", partition_0 "
"from gnomad.sites " 
"where cardinality(filters) = 0")

**gnomAD view**

In [205]:
query = "select * from gnomad limit 10"
df = execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 1110.29305
        ExecutionTime(s): 2.524
        QueuingTime(s): 0.237


,variant_id,chrom,pos,ref,alt,rsid,filters,info_ac,info_an,info_af,info_popmax,partition_0
0,chr18:A:G:856887,chr18,856887,A,G,rs1403391816,[],[2],152092,[1.31499E-5],[nfe],chr18
1,chr18:G:C:856899,chr18,856899,G,C,None,[],[1],152130,[6.57333E-6],[nfe],chr18
2,chr18:G:C:856903,chr18,856903,G,C,rs1398081239,[],[1],152098,[6.57471E-6],[eas],chr18
3,chr18:G:A:856905,chr18,856905,G,A,rs1337701794,[],[1],152082,[6.5754E-6],[eas],chr18
4,chr18:C:T:856909,chr18,856909,C,T,rs1400157635,[],[3],152116,[1.97218E-5],[eas],chr18
5,chr18:C:T:856910,chr18,856910,C,T,None,[],[1],152156,[6.5722E-6],[eas],chr18
6,chr18:A:G:856919,chr18,856919,A,G,None,[],[1],152110,[6.57419E-6],None,chr18
7,chr18:A:G:856923,chr18,856923,A,G,None,[],[1],152104,[6.57445E-6],[afr],chr18
8,chr18:A:G:856929,chr18,856929,A,G,rs1342158649,[],[2],152094,[1.31498E-5],[nfe],chr18
9,chr18:G:A:856932,chr18,856932,G,A,rs1310906294,[],[1],152100,[6.57462E-6],[amr],chr18


**Example 6:** Find all rare variants (Minor Allele frequency < 0.01) in the BRCA1 gene from gnomAD. We will use the view we created above.

In [206]:
query = "SELECT COUNT(*) from gnomad \
         WHERE partition_0 = 'chr17' \
         AND (cardinality(info_af) > 0 and info_af[1] < 0.01) \
         AND pos BETWEEN 43044295 and 43125364"
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 3.43072
        ExecutionTime(s): 2.562
        QueuingTime(s): 0.183


,_col0
0,16938


**Example 7:** Let us now find how many samples in the 1000 genomes dataset have rare variants (at least 2 subjects with Minor allele frequency < 0.01) in the BRCA1 region.

In [207]:
query = """SELECT COUNT(DISTINCT sample_id) FROM {}.{} as v 
           JOIN gnomad AS a  
           ON v.variant_id = a.variant_id 
           WHERE v.chrom='chr17' AND
           v.pos BETWEEN 43044295 AND 43125364 AND
           cardinality(a.info_af) > 0 AND a.info_af[1] < 0.01 
           AND a.info_an > 1 """.format(databasename,partitioned_chr)
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 145584.90366
        ExecutionTime(s): 27.747
        QueuingTime(s): 0.163


,_col0
0,3068


**Example 7a:** Find all samples in the 1000 genomes dataset have rare variants (at least 2 subjects with Minor allele frequency < 0.01) in the BRCA1 region and are labeled Pathogenic or Likely pathogenic in ClinVar.

In [210]:
query = """SELECT * FROM {}.{} as v 
           JOIN gnomad AS a  
           ON v.variant_id = a.variant_id 
           JOIN clinvar as clin
           on a.variant_id = clin.variant_id
           WHERE v.chrom='chr17' AND
           v.pos BETWEEN 43044295 AND 43125364 AND
           cardinality(filter(info_af, x -> x < 0.01)) > 0 
           AND a.info_an > 1 
           AND (clin.clinicalsignificance ='Pathogenic' OR
                clin.clinicalsignificance = 'Likely pathogenic')""".format(databasename,partitioned_chr)
df=execute_query_async(query)
df

Query execution summary:
        DataScanned(MB): 406273.40489
        ExecutionTime(s): 38.644
        QueuingTime(s): 0.261


,variant_id,pos,ref,alt,sample_id,alleles,rsid,qual,filters,info.ac,...,chrom,pos,ref,alt,genename,clinicalsignificance,num_submitters,reviewstatus,phenotypeids,phenotypelist
0,chr17:C:T:43082403,43082403,C,T,HG00365,"[C, T]",None,50.00,[],[1],...,17,43082403,C,T,BRCA1,Pathogenic,16,reviewed by expert panel,"[MONDO:MONDO:0011450, MedGen:C2676676, OMIM:60...","[Breast-ovarian cancer, familial 1|not provid..."
1,chr17:G:A:43092128,43092128,G,A,HG03929,"[G, A]",None,49.67,[],[1],...,17,43092128,G,A,BRCA1,Pathogenic,11,reviewed by expert panel,"[MONDO:MONDO:0011450, MedGen:C2676676, OMIM:60...","[Breast-ovarian cancer, familial 1|not provid..."
2,chr17:G:A:43124063,43124063,G,A,HG02164,"[G, A]",None,50.00,[],[1],...,17,43124063,G,A,BRCA1,Pathogenic,14,reviewed by expert panel,"[MONDO:MONDO:0011450, MedGen:C2676676, OMIM:60...","[Breast-ovarian cancer, familial 1|not provid..."


In [211]:
cursor.close()
conn.close()